In [1]:
import os
import sys 
import gc
import  numpy as np
import pandas as pd
import patsy as pt
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
matplotlib.use('TkAgg')
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan #BP检验
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# 读取数据

In [2]:
dataset_path = "datasets/train/"
predict_data_path = "datasets/test_example.csv"

In [3]:
feats_uscols = ["F"+str(i) for i in range(1,33)]
uscols = ["Y"] + feats_uscols
dfs = []

In [4]:
for j in range(70):
# for j in range(10):
    path = dataset_path + "data.{}.csv".format(j)
    dfs.append(pd.read_csv(path, delimiter=',', usecols=uscols))

In [5]:
# df_data.head()

In [6]:
# df_data = pd.concat(dfs)
# df_train, df_test = train_test_split(df_data, test_size=0.2, random_state=0)

In [7]:
df_train, df_test = pd.concat(dfs[:60]), pd.concat(dfs[60:])

In [8]:
df_example = pd.read_csv(predict_data_path, delimiter=',', usecols=feats_uscols)

In [9]:
# df_example.head()

# 模型1：RF

In [10]:
def prediction(df_train, df_test, X_predict):
    # X_train = df_train.drop(columns={'Y', 'C'})
    X_train = df_train.drop(columns={'Y'})
    y_train = df_train['Y']

    # X_test = df_test.drop(columns={'Y', 'C'})
    X_test = df_test.drop(columns={'Y'})
    y_test = df_test['Y']

    # forest = RandomForestRegressor(n_estimators=1000, criterion='squared_error', random_state=1, n_jobs=-1)
    # forest = RandomForestRegressor(n_estimators=100, criterion='squared_error', random_state=1, n_jobs=-1)
    forest = RandomForestRegressor(n_estimators=10, criterion='squared_error', random_state=1, n_jobs=-1)
    forest.fit(X_train, y_train)

    p_test = forest.predict(X_test)

    mse = mean_squared_error(p_test, y_test)

    print("mse: ", mse)
    print("rmse: ", np.sqrt(mse))

    # X_predict = X_predict.drop(columns={'C'})
    y_test_pred = forest.predict(X_predict)

    return y_test_pred

In [11]:
res = prediction(df_train, df_test, df_example)
res 

mse:  0.06128938601236784
rmse:  0.2475669323887337


array([-0.0753,  0.0097])

# 模型2：lgb

In [12]:
df_train.fillna(df_train.median(),inplace = True)
x_train = df_train.drop(['F17','Y'], axis=1)
y_train = df_train['Y'].values
x_example = df_example.drop(['F17'], axis=1)
print(x_train.shape, y_train.shape)

(834286, 31) (834286,)


In [13]:
# del df_train; gc.collect()
x_train = x_train.values.astype(np.float32, copy=False)
d_train = lgb.Dataset(x_train, label=y_train)

In [14]:
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          
params['sub_feature'] = 0.5      
params['bagging_fraction'] = 0.85 
params['bagging_freq'] = 40
params['num_leaves'] = 512        
params['min_data'] = 500         
params['min_hessian'] = 0.05     
params['verbose'] = 0

print("Fitting LightGBM model\n")


Fitting LightGBM model



In [15]:
#model = lgb.train(params, d_train, 430)
model = lgb.Booster(model_file='lgbm_model.mdl')
#model.save_model('lgbm_model.mdl')

In [26]:
# path = "./train/data.69.csv"
# path = "../yujin/train/train/data.69.csv"
# df_test = pd.read_csv(path, delimiter=',', usecols=uscols)
x_test = df_test.drop(['F17','Y'], axis=1)
y_test = df_test['Y'].values
p_test = model.predict(x_test)
# del x_test; gc.collect()

mse = mean_squared_error(p_test, y_test)

print("mse: ", mse)
print("rmse: ", np.sqrt(mse))

mse:  0.055099930701335856
rmse:  0.23473374427494623


In [17]:
model.predict(x_example)

array([-0.00531805,  0.00087267])

# 模型3: fwls 

In [24]:
def prediction(data_example, df_test, X_test):
    ## train 
    X_list = '+'.join(data_example.columns[1:-1])
    # OLS:
    reg_ols = smf.ols(formula='Y ~ ' + X_list, data=data_example)
    results_ols = reg_ols.fit()
    
    # log(残差平方)对自变量做回归
    data_example['loge2'] = np.log(results_ols.resid ** 2)

    reg_loge2 = smf.ols(formula='loge2 ~ ' + X_list, data=data_example)
    # reg_loge2 = smf.ols(formula='loge2 ~ ' + X_list, data=data_train)

    results_loge2 = reg_loge2.fit()

    # FWLS
    wls_weight = list(1 / np.exp(results_loge2.fittedvalues))
    reg_wls = smf.wls(formula='Y ~ ' + X_list, weights=wls_weight, data=data_example)
    # reg_wls = smf.wls(formula='Y ~ ' + X_list, weights=wls_weight, data=data_train)

    results_wls = reg_wls.fit()

    
    ## test 
    data_test_x = df_test.drop(columns = {'Y'})
    data_test_y = df_test['Y']
    
    p_test = results_wls.predict(data_test_x)
    mse = mean_squared_error(p_test, data_test_y)

    print("mse: ", mse)
    print("rmse: ", np.sqrt(mse))

    prediction_value = results_wls.predict(X_test)
    return prediction_value

In [25]:
res = prediction(df_train, df_test, X_test=df_example) 


mse:  0.055558880999184976
rmse:  0.23570931462117695
